In [ ]:
#each node has the form [parent, player or payoff, label, optionally probability]
#3 examples are provided below immediately
#nodes = [[-1, 0, -1], [0, [0, 100], 'dont enter'], [0, 1, 'enter'], [2, [50, 50], 'dont challenge'], [2, 0, 'challenge'], [4, [0, 70], 'backoff'], [4, [30, 60], 'dont backoff']]
#nodes = [[-1, 0, -1], [0, 1, 'L'], [0, 1, 'R'], [1, [0, 1], 'L'], [1, [1, 2], 'R'], [2, [2, 3], 'L'], [2, [3, 4], 'R']]
nodes = [[-1, 0, -1], [0, 1, 'stay in'], [0, 'N', 'go out'], [1, [0, 2], 'disturb'], [1, [1, 1], 'dont disturb'], [2, 1, 'rain', 0.3], [2, 1, 'no rain', 0.7], [5, [-2, 1], 'disturb'], [5, [-1, 0], 'dont disturb'], [6, [2, 2], 'play'], [6, [0, 0], 'dont play']]
print(len(nodes))

###########################################################

#We define this to be able to output the SPNE at the end, which is called "gameplay" here
#This is a vector of ordered pairs with the first element being the node number, and the second element being the chosen action at that node
#We fill the action to be '' for every node here, we will modify that after the SPNE is found out

gamePlay = []

for i in range(len(nodes)):
    gamePlay.insert(len(gamePlay), [nodes[i][1], ''])

print(gamePlay)

###########################################################

#We solve the game through backward induction, but at each level, we begin with the rightmost node first
#We have to make certain changes to the methods however: beginning the backward induction at a node that is not at the maximum distance to the initial node would make payof calculations complicated
#Therefore, when doing the backward induction, we begin with a node that is at the maximum possible distance from the initial node
#This guarantees that all the children of its parent are terminal nodes

for i in range(len(nodes)):

    #We begin with the last node, find its parent node, the parent node's player
    
    parNode = len(nodes) - i - 1
    parPlayer = nodes[parNode][1]
    childrenNodes = []
    childrenPayoffs = []
    childrenProbabilities = []
    
    #If the parent player is not nature, we find its other children through a linear search and add them to a vector
    
    if type(parPlayer) is int:
        for j in range(parNode + 1, len(nodes)):
            if nodes[j][0] == parNode:
                childrenNodes.insert(len(childrenNodes), j)
                if type(nodes[j][1]) is list:
                    childrenPayoffs.insert(len(childrenPayoffs), nodes[j][1][parPlayer])
                
                #Note that if our code works well, this conditional should NEVER be satisfied, as all the children of the parent are supposed to be terminal nodes
                #Thus, this acts only as a sanity check and does not affect the rest of the code
                #The reader can ignore it
                
                if type(nodes[j][1]) is int:
                    childrenPayoffs.insert(len(childrenPayoffs), -100000000000) #to edit later
        
        #We now find the action that yields the highest payoff
        
        optimalPayoffIndex = childrenPayoffs.index(max(childrenPayoffs))
        optimalChoice = nodes[childrenNodes[optimalPayoffIndex]]
        
        #We add that to the gamePlay data
        #We also replace the 'player' of the parent with the payoff vector associated with the relevant player's optimal choice
        #This converts the parent to a terminal node too and ensures that the backward induction deals only with nodes whose parents have terminal nodes as children
        
        gamePlay[parNode][1] = optimalChoice[2]
        nodes[parNode][1] = optimalChoice[1]
     
    #If the player of the parent node is nature, we find the parent node's children, and the expected payoff of each player
    #Note that each child is again a terminal node, due to arguments presented earlier
    
    if parPlayer == 'N':
        for j in range(parNode + 1, len(nodes)):
            if nodes[j][0] == parNode:
                childrenNodes.insert(len(childrenNodes), j)
                childrenPayoffs.insert(len(childrenPayoffs), nodes[j][1])
                childrenProbabilities.insert(len(childrenNodes), nodes[j][3])
        expectedPayoff = []
        for j in range(len(childrenPayoffs[0])):
            ep = 0
            for k in range(len(childrenNodes)):
                ep += childrenPayoffs[k][j] * childrenProbabilities[k]
            expectedPayoff.insert(len(expectedPayoff), ep)
        nodes[parNode][1] = expectedPayoff
        
#print(gamePlay) #We can print the gameplay directly but that looks a bit disorganised

#We print the gameplay like this instead for clarity

for i in range(len(gamePlay)):
    if type(gamePlay[i][0]) is int:
        print(gamePlay[i])

###############################################################################